<a href="https://colab.research.google.com/github/jacklazer/Speedup-para-aplicaci-n-h-brida-Python-C---2/blob/main/vectorScalarMultiply.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Integrantes:
- Hernan David Cisneros Vargas
- Juan Sebastian Cifuentes
- Santiago Duque Chacón

In [42]:
!apt-get update
!apt-get install gcc


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gcc is already the newest version (4:11.2.0-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


In [43]:
%%writefile vector_scalar_multiply.c

#include <immintrin.h>
#include <stdio.h>
#include <assert.h>

#define VECTORSIZE 4


// Function to perform vector-scalar multiplication
void vectorScalarMultiply(const double* vector, double scalar, double* result, int length) {
    // Ensure the length is a multiple of 4 for proper alignment
    int alignedLength = (length + 3) & ~3;

    // Loop through the vector in 4-element chunks
    for (int i = 0; i < alignedLength; i += 4) {
        // Load the vector chunk into AVX register
        __m256d vec = _mm256_loadu_pd(vector + i);

        // Broadcast the scalar value to all elements of another AVX register
        __m256d scalarVec = _mm256_broadcast_sd(&scalar);

        // Perform element-wise multiplication
        __m256d resultVec = _mm256_mul_pd(vec, scalarVec);

        // Store the result back to memory
        _mm256_storeu_pd(result + i, resultVec);
    }
}




Overwriting vector_scalar_multiply.c


In [44]:
!rm -f libvector.so
!gcc -shared -o libvector.so -fPIC -mavx vector_scalar_multiply.c

In [7]:
import ctypes
import numpy as np

# Cargando la biblioteca
lib = ctypes.CDLL('./libvector.so')

# Definiendo los tipos de argumentos y valor de retorno para la función en C
lib.vectorScalarMultiply.argtypes = [np.ctypeslib.ndpointer(dtype=np.double),
                                      ctypes.c_double,
                                      np.ctypeslib.ndpointer(dtype=np.double),
                                      ctypes.c_int]
lib.vectorScalarMultiply.restype = None


In [36]:
import timeit
import time
import cProfile as profile
import pstats

vector = np.random.rand(5000000)
scalar = 3.0
result = np.zeros_like(vector)
prof = profile.Profile()

#inicio captura de tiempo de ejecucion ingenuo
inicio = time.time()

#inicio captura de tiempo de ejecución del programa en libreria de enlace dinamico
starting_time = timeit.default_timer()

#inicio captura del perfilamiento
prof.enable()

lib.vectorScalarMultiply(vector, scalar, result, len(vector))

#fin captura del perfilamiento
prof.disable()

#fin captura de tiempo de ejecución del programa en libreria de enlace dinamico
ending_time = timeit.default_timer()

#fin captura de tiempo de ejecucion ingenuo
fin = time.time()

print("Resultado multiplicacion vectorial:", result, "\n\n")
print(f"Tiempo transcurrido (ingenuo) {fin - inicio}\n") # f-strings
print(f"Tiempo transcurrido (timeit) {ending_time - starting_time}\n") # f-strings
print(f"Tiempo transcurrido (cprofile)\n")
stats = pstats.Stats(prof).strip_dirs().sort_stats("cumtime")
stats.print_stats(10)


Resultado multiplicacion vectorial: [1.79796459 0.01606188 0.40795824 ... 0.62388416 0.48815062 2.54463064] 


Tiempo transcurrido (ingenuo) 0.012319803237915039

Tiempo transcurrido (timeit) 0.012193608999950811

Tiempo transcurrido (cprofile)

         102 function calls in 0.012 seconds

   Ordered by: cumulative time
   List reduced from 35 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.012    0.004 interactiveshell.py:3512(run_code)
        3    0.000    0.000    0.012    0.004 {built-in method builtins.exec}
        1    0.012    0.012    0.012    0.012 <ipython-input-36-d92dbda146be>:1(<cell line: 20>)
        3    0.000    0.000    0.000    0.000 codeop.py:117(__call__)
        3    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        3    0.000    0.000    0.000    0.000 interactiveshell.py:3337(_update_code_co_name)
        2    0.000    0.000    0.000    0.000 ctyp